In [1]:
pip install deap update_checker tqdm stopit xgboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached stopit-1.1.2-py3-none-any.whl
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Created wheel for deap: filename=deap-1.4.1-cp310-cp310-macosx_14_0_arm64.whl size=104428 sha256=ff4034d3efb67d65bddd2f966a87bf7e817d51734a6ad4265037758b8de5e74c
  Stored in directory: /Users/youssefelmaaoua/Library/Caches/pip/wheels/0e/f2/e3/f1bd8b40cf7eefa319d5a27aa9ba3cc588e6ada69b5919590d
Successfully built deap
Note: you may need to restart the kernel to use updated packages.


In [1]:
! pip install tpot


In [4]:
pip install torch


  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 5.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy scipy scikit-learn pandas joblib pytorch


  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pytorch (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Traceback (most recent call last):
        File "/Users/youssefelmaaoua/.pyenv/versions/3.10.6/envs/Deep_fake_voice_recognition/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/Users/youssefelmaaoua/.pyenv/versions/3.10.6/envs/Deep_fake_voice_recognition/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "/Users/youssefelmaaoua/.pyenv/versions/3.10.6/envs/Deep_fake_voice_recognition/lib/python3.10/site-packages/pip/_vendor/pypr

In [35]:
from tpot import TPOTClassifier
import joblib


In [10]:
import os


In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc
from sklearn.ensemble import RandomForestClassifier


In [36]:
def get_train_test(complete_df):
    #split, process and return  X_train, X_test, y_train, y_test
    df_train, df_test = train_test_split(complete_df, test_size=0.3,stratify=complete_df['LABEL'])

    count = df_train['LABEL'].value_counts()
    count[count.isin([min(count)])].index[0]
    under_sampled = df_train[df_train['LABEL']==count[count.isin([max(count)])].index[0]].sample(min(count))
    balanced_df = pd.concat([under_sampled, df_train[df_train['LABEL']==count[count.isin([min(count)])].index[0]]], axis=0)
    X_train = balanced_df[balanced_df.columns[0:-1]]
    X_test = df_test[balanced_df.columns[0:-1]]
    y_label = balanced_df['LABEL']
    y_label_test = df_test['LABEL']
    le = LabelEncoder()
    le.fit(y_label)
    # le.classes_
    y_train = le.transform(y_label)
    y_test = le.transform(y_label_test)
    return X_train, X_test, y_train, y_test


In [37]:
def test_model(svm_XGB_best2):
    #test model with our tests

    #read tests
    new_test_taylor_r = pd.read_csv('Data/extracted_segment_10_extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.csv')
    new_test_bidenf = pd.read_csv('Data/audio_features_biden_AI.csv')
    new_test_bidenr = pd.read_csv('Data/audio_features_biden.csv')
    new_test_el = pd.read_csv('Data/Elise.csv')
    new_test_yr1 = pd.read_csv('Data/Youssef.csv')
    new_test_yr2 = pd.read_csv('Data/Youssef_2.csv')
    new_test_yf1 = pd.read_csv('Data/Youssef_Eric Cartman.csv')
    new_test_yf2 = pd.read_csv('Data/Youssef_2_Female.csv')
    new_test_yf3  = pd.read_csv('Data/Youssef_2_Male Reggaeton.csv')
    new_test_MF_f  = pd.read_csv('Data/Morgan Freeman_fake.csv')
    new_test_MF_r  = pd.read_csv('Data/Morgan Freeman_real.csv')
    #make y_true
    y_real = np.ones((1000,))
    y_fake = np.zeros((1000,))

    #predict and get accuray score
    pred = svm_XGB_best2.predict(new_test_taylor_r)
    taylor_r = accuracy_score(y_real[:len(pred)],pred)
    print('Real Taylor : ' , taylor_r)

    pred = svm_XGB_best2.predict(new_test_bidenf[new_test_bidenf.columns[0:-2]])
    bidenf = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Biden : ' , bidenf)

    pred = svm_XGB_best2.predict(new_test_bidenr[new_test_bidenf.columns[0:-2]])
    bidenr=accuracy_score(y_real[:len(pred)],pred)
    print('Real Biden : ' , bidenr)

    pred = svm_XGB_best2.predict(new_test_el)
    eliser=accuracy_score(y_real[:len(pred)],pred)
    print('Real Elise : ' , eliser)

    pred = svm_XGB_best2.predict(new_test_yr1)
    youssr1 = accuracy_score(y_real[:len(pred)],pred)
    print('Real Youss_1 : ' , youssr1)

    pred = svm_XGB_best2.predict(new_test_yr2)
    youssr2 = accuracy_score(y_real[:len(pred)],pred)
    print('Real Youss_2 : ' , youssr2)

    pred = svm_XGB_best2.predict(new_test_yf1)
    youssf1 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_1 : ' , youssf1)

    pred = svm_XGB_best2.predict(new_test_yf2)
    youssf2 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_2 : ' , youssf2)

    pred = svm_XGB_best2.predict(new_test_yf3)
    youssf3 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_3 : ' , youssf3)

    pred = svm_XGB_best2.predict(new_test_MF_f)
    morganf = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Morgan : ' , morganf)

    pred = svm_XGB_best2.predict(new_test_MF_r)
    morganr = accuracy_score(y_real[:len(pred)],pred)
    print('Real Morgan : ' , morganr)


In [47]:
pipeline_optimizer = TPOTClassifier(n_jobs=-1,warm_start=True,verbosity=2)


In [12]:
os.cpu_count()


8

In [19]:
!pip install GPUtil


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=f19050c02cb2f96bf347631992ddc45e74140791c335109b9e1e318930b47ebd
  Stored in directory: /Users/youssefelmaaoua/Library/Caches/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [20]:
import GPUtil
GPUtil.getAvailable()


[]

In [21]:
import torch

torch.cuda.is_available()


False

In [43]:
pwd


'/Users/youssefelmaaoua/code/Andreanorm/Deep_fake_voice_recognition'

In [46]:
df = pd.read_csv('Data/df_asvc_without_LA.csv')


In [51]:
X_train, X_test, y_train, y_test = get_train_test(df)


In [52]:
pipeline_optimizer.fit(X_train, y_train)


Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.001, max_depth=10, min_child_weight=3, n_estimators=100, n_jobs=1, subsample=0.2, verbosity=0)


TPOTClassifier(n_jobs=7, verbosity=2, warm_start=True)

In [26]:
pipeline_optimizer


TPOTClassifier(n_jobs=7)

In [53]:
print(pipeline_optimizer.score(X_test, y_test))


0.6663023226928332


In [31]:
pipeline_optimizer.export('tpot_exported_pipeline_withoutLA_v2.py')


In [54]:
import pickle
with open('./test_tpot_notebook.pkl','wb') as f:
    pickle.dump(pipeline_optimizer.fitted_pipeline_, f)
